In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Masking
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize

"""
    Discrete log-likelihood for Weibull hazard function on censored survival data
    y_true is a (samples, 2) tensor containing time-to-event (y), and an event indicator (u)
    ab_pred is a (samples, 2) tensor containing predicted Weibull alpha (a) and beta (b) parameters
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)

    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

"""
    Not used for this model, but included in case somebody needs it
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))


"""
    Custom Keras activation function, outputs alpha neuron using exponentiation and beta using softplus
"""
def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])

    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))

    return k.concatenate((a, b), axis=1)


"""
    Load and parse engine data files into:
       - an (engine/day, observed history, sensor readings) x tensor, where observed history is 100 days, zero-padded
         for days that don't have a full 100 days of observed history (e.g., first observed day for an engine)
       - an (engine/day, 2) tensor containing time-to-event and 1 (since all engines failed)

    There are probably MUCH better ways of doing this, but I don't use Numpy that much, and the data parsing isn't the
    point of this demo anyway.
"""
def load_file(name):
    with open(name, 'r') as file:
        return np.loadtxt(file, delimiter=',')

np.set_printoptions(suppress=True, threshold=10000)

train = load_file('train.csv')
test_x = load_file('test_x.csv')
test_y = load_file('test_y.csv')

# Combine the X values to normalize them, then split them back out
all_x = np.concatenate((train[:, 2:26], test_x[:, 2:26]))
all_x = normalize(all_x, axis=0)

train[:, 2:26] = all_x[0:train.shape[0], :]
test_x[:, 2:26] = all_x[train.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test_x[:, 0:2] -= 1

# Configurable observation look-back period for each engine/day
max_time = 100

def build_data(engine, time, x, max_time, is_test):
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = np.empty((0, 2), dtype=np.float32)

    # A full history of sensor readings to date for each x
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(100):
        print("Engine: " + str(i))
        # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
        max_engine_time = int(np.max(time[engine == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = x[engine == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, 1), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

train_x, train_y = build_data(train[:, 0], train[:, 1], train[:, 2:26], max_time, False)
test_x = build_data(test_x[:, 0], test_x[:, 1], test_x[:, 2:26], max_time, True)[0]

train_u = np.zeros((100, 1), dtype=np.float32)
train_u += 1
test_y = np.append(np.reshape(test_y, (100, 1)), train_u, axis=1)

"""
    Here's the rest of the meat of the demo... actually fitting and training the model.
    We'll also make some test predictions so we can evaluate model performance.
"""

# Start building our model
model = Sequential()

# Mask parts of the lookback period that are all zeros (i.e., unobserved) so they don't skew the model
model.add(Masking(mask_value=0., input_shape=(max_time, 24)))

# LSTM is just a common type of RNN. You could also try anything else (e.g., GRU).
model.add(LSTM(20, input_dim=24))

# We need 2 neurons to output Alpha and Beta parameters for our Weibull distribution
model.add(Dense(2))

# Apply the custom activation function mentioned above
model.add(Activation(activate))

# Use the discrete log-likelihood for Weibull survival data as our loss function
model.compile(loss=weibull_loglik_discrete, optimizer=RMSprop(lr=.001))

# Fit!
model.fit(train_x, train_y, nb_epoch=250, batch_size=2000, verbose=2, validation_data=(test_x, test_y))

# Make some predictions and put them alongside the real TTE and event indicator values
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (100, 2))
test_result = np.concatenate((test_y, test_predict), axis=1)

# TTE, Event Indicator, Alpha, Beta
print(test_result)


Using TensorFlow backend.


Engine: 0
Engine: 1
Engine: 2
Engine: 3
Engine: 4
Engine: 5
Engine: 6
Engine: 7
Engine: 8
Engine: 9
Engine: 10
Engine: 11
Engine: 12
Engine: 13
Engine: 14
Engine: 15
Engine: 16
Engine: 17
Engine: 18
Engine: 19
Engine: 20
Engine: 21
Engine: 22
Engine: 23
Engine: 24
Engine: 25
Engine: 26
Engine: 27
Engine: 28
Engine: 29
Engine: 30
Engine: 31
Engine: 32
Engine: 33
Engine: 34
Engine: 35
Engine: 36
Engine: 37
Engine: 38
Engine: 39
Engine: 40
Engine: 41
Engine: 42
Engine: 43
Engine: 44
Engine: 45
Engine: 46
Engine: 47
Engine: 48
Engine: 49
Engine: 50
Engine: 51
Engine: 52
Engine: 53
Engine: 54
Engine: 55
Engine: 56
Engine: 57
Engine: 58
Engine: 59
Engine: 60
Engine: 61
Engine: 62
Engine: 63
Engine: 64
Engine: 65
Engine: 66
Engine: 67
Engine: 68
Engine: 69
Engine: 70
Engine: 71
Engine: 72
Engine: 73
Engine: 74
Engine: 75
Engine: 76
Engine: 77
Engine: 78
Engine: 79
Engine: 80
Engine: 81
Engine: 82
Engine: 83
Engine: 84
Engine: 85
Engine: 86
Engine: 87
Engine: 88
Engine: 89
Engine: 90
Engine: 9

C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:139: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:139: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, input_shape=(None, 24))`
C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:151: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20631 samples, validate on 100 samples
Epoch 1/250
 - 4s - loss: 21.8059 - val_loss: 13.0132
Epoch 2/250
 - 3s - loss: 11.4287 - val_loss: 7.1513
Epoch 3/250
 - 3s - loss: 8.6117 - val_loss: 7.1352
Epoch 4/250
 - 3s - loss: 8.3966 - val_loss: 7.0938
Epoch 5/250
 - 3s - loss: 8.1951 - val_loss: 7.0504
Epoch 6/250
 - 3s - loss: 8.0114 - val_loss: 7.0019
Epoch 7/250
 - 3s - loss: 7.8390 - val_loss: 6.9433
Epoch 8/250
 - 3s - loss: 7.6716 - val_loss: 6.8594
Epoch 9/250
 - 3s - loss: 7.5009 - val_loss: 6.7318
Epoch 10/250
 - 3s - loss: 7.3132 - val_loss: 6.5489
Epoch 11/250
 - 3s - loss: 7.0943 - val_loss: 6.3254
Epoch 12/250
 - 3s - loss: 6.7998 - val_loss: 5.9648
Epoch 13/250
 - 3s - loss: 6.4426 - val_loss: 5.6191
Epoch 14/250
 - 3s - loss: 6.1197 - val_loss: 5.3702
Epoch 15/250
 - 3s - loss: 5.8838 - val_loss: 5.2422
Epoch 16/250
 - 3s - loss: 5.7653 - val_loss: 5.2455
Epoch 17/250
 - 3s - loss: 5.7141 - val_loss: 5.1706
Epoch 18/250
 - 3s - loss: 5.6979 - val_loss: 5.1858
Epoc

Epoch 154/250
 - 3s - loss: 5.2639 - val_loss: 5.0670
Epoch 155/250
 - 3s - loss: 5.3069 - val_loss: 5.0934
Epoch 156/250
 - 3s - loss: 5.2838 - val_loss: 5.0847
Epoch 157/250
 - 3s - loss: 5.2765 - val_loss: 5.0665
Epoch 158/250
 - 3s - loss: 5.2946 - val_loss: 5.0989
Epoch 159/250
 - 3s - loss: 5.2620 - val_loss: 5.1137
Epoch 160/250
 - 3s - loss: 5.2921 - val_loss: 5.0910
Epoch 161/250
 - 3s - loss: 5.2858 - val_loss: 5.1207
Epoch 162/250
 - 4s - loss: 5.3096 - val_loss: 5.1149
Epoch 163/250
 - 4s - loss: 5.2533 - val_loss: 5.0638
Epoch 164/250
 - 3s - loss: 5.2642 - val_loss: 5.1018
Epoch 165/250
 - 3s - loss: 5.2785 - val_loss: 5.1294
Epoch 166/250
 - 3s - loss: 5.2659 - val_loss: 5.0807
Epoch 167/250
 - 3s - loss: 5.2917 - val_loss: 5.1122
Epoch 168/250
 - 3s - loss: 5.3094 - val_loss: 5.0972
Epoch 169/250
 - 3s - loss: 5.2572 - val_loss: 5.1161
Epoch 170/250
 - 3s - loss: 5.2682 - val_loss: 5.0653
Epoch 171/250
 - 3s - loss: 5.3390 - val_loss: 5.0894
Epoch 172/250
 - 3s - loss: 

In [4]:
train_y

array([[192.,   1.],
       [191.,   1.],
       [190.,   1.],
       ...,
       [  3.,   1.],
       [  2.,   1.],
       [  1.,   1.]])

In [2]:

"""
    Discrete log-likelihood for Weibull hazard function on censored survival data
    y_true is a (samples, 2) tensor containing time-to-event (y), and an event indicator (u)
    ab_pred is a (samples, 2) tensor containing predicted Weibull alpha (a) and beta (b) parameters
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)

    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

"""
    Not used for this model, but included in case somebody needs it
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))


"""
    Custom Keras activation function, outputs alpha neuron using exponentiation and beta using softplus
"""
def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])

    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))

    return k.concatenate((a, b), axis=1)


"""
    Load and parse engine data files into:
       - an (engine/day, observed history, sensor readings) x tensor, where observed history is 100 days, zero-padded
         for days that don't have a full 100 days of observed history (e.g., first observed day for an engine)
       - an (engine/day, 2) tensor containing time-to-event and 1 (since all engines failed)

    There are probably MUCH better ways of doing this, but I don't use Numpy that much, and the data parsing isn't the
    point of this demo anyway.
"""
def load_file(name):
    with open(name, 'r') as file:
        return np.loadtxt(file, delimiter=',')

np.set_printoptions(suppress=True, threshold=10000)

train = load_file('train.csv')
test_x = load_file('test_x.csv')
test_y = load_file('test_y.csv')

# Combine the X values to normalize them, then split them back out
all_x = np.concatenate((train[:, 2:26], test_x[:, 2:26]))
all_x = normalize(all_x, axis=0)

train[:, 2:26] = all_x[0:train.shape[0], :]
test_x[:, 2:26] = all_x[train.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test_x[:, 0:2] -= 1

# Configurable observation look-back period for each engine/day
max_time = 100

def build_data(engine, time, x, max_time, is_test):
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = np.empty((0, 2), dtype=np.float32)

    # A full history of sensor readings to date for each x
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(100):
        print("Engine: " + str(i))
        # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
        max_engine_time = int(np.max(time[engine == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = x[engine == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, 1), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

train_x, train_y = build_data(train[:, 0], train[:, 1], train[:, 2:26], max_time, False)
test_x = build_data(test_x[:, 0], test_x[:, 1], test_x[:, 2:26], max_time, True)[0]

train_u = np.zeros((100, 1), dtype=np.float32)
train_u += 1
test_y = np.append(np.reshape(test_y, (100, 1)), train_u, axis=1)

"""
    Here's the rest of the meat of the demo... actually fitting and training the model.
    We'll also make some test predictions so we can evaluate model performance.
"""

Engine: 0
Engine: 1
Engine: 2
Engine: 3
Engine: 4
Engine: 5
Engine: 6
Engine: 7
Engine: 8
Engine: 9
Engine: 10
Engine: 11
Engine: 12
Engine: 13
Engine: 14
Engine: 15
Engine: 16
Engine: 17
Engine: 18
Engine: 19
Engine: 20
Engine: 21
Engine: 22
Engine: 23
Engine: 24
Engine: 25
Engine: 26
Engine: 27
Engine: 28
Engine: 29
Engine: 30
Engine: 31
Engine: 32
Engine: 33
Engine: 34
Engine: 35
Engine: 36
Engine: 37
Engine: 38
Engine: 39
Engine: 40
Engine: 41
Engine: 42
Engine: 43
Engine: 44
Engine: 45
Engine: 46
Engine: 47
Engine: 48
Engine: 49
Engine: 50
Engine: 51
Engine: 52
Engine: 53
Engine: 54
Engine: 55
Engine: 56
Engine: 57
Engine: 58
Engine: 59
Engine: 60
Engine: 61
Engine: 62
Engine: 63
Engine: 64
Engine: 65
Engine: 66
Engine: 67
Engine: 68
Engine: 69
Engine: 70
Engine: 71
Engine: 72
Engine: 73
Engine: 74
Engine: 75
Engine: 76
Engine: 77
Engine: 78
Engine: 79
Engine: 80
Engine: 81
Engine: 82
Engine: 83
Engine: 84
Engine: 85
Engine: 86
Engine: 87
Engine: 88
Engine: 89
Engine: 90
Engine: 9

"\n    Here's the rest of the meat of the demo... actually fitting and training the model.\n    We'll also make some test predictions so we can evaluate model performance.\n"

In [3]:

# Start building our model
model = Sequential()

# Mask parts of the lookback period that are all zeros (i.e., unobserved) so they don't skew the model
model.add(Masking(mask_value=0., input_shape=(max_time, 24)))

# LSTM is just a common type of RNN. You could also try anything else (e.g., GRU).
model.add(LSTM(20, input_dim=24))

# We need 2 neurons to output Alpha and Beta parameters for our Weibull distribution
model.add(Dense(2))

# Apply the custom activation function mentioned above
model.add(Activation(activate))

# Use the discrete log-likelihood for Weibull survival data as our loss function
model.compile(loss=weibull_loglik_discrete, optimizer=RMSprop(lr=.001))

# Fit!
model.fit(train_x, train_y, nb_epoch=250, batch_size=2000, verbose=2, validation_data=(test_x, test_y))

# Make some predictions and put them alongside the real TTE and event indicator values
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (100, 2))
test_result = np.concatenate((test_y, test_predict), axis=1)

# TTE, Event Indicator, Alpha, Beta
print(test_result)


C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, input_shape=(None, 24))`
  
C:\Users\mromanova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20631 samples, validate on 100 samples
Epoch 1/250
 - 4s - loss: 22.1831 - val_loss: 13.8559
Epoch 2/250
 - 3s - loss: 11.9478 - val_loss: 7.2581
Epoch 3/250
 - 3s - loss: 8.7455 - val_loss: 7.2005
Epoch 4/250
 - 3s - loss: 8.4764 - val_loss: 7.1438
Epoch 5/250
 - 3s - loss: 8.2479 - val_loss: 7.0768
Epoch 6/250
 - 3s - loss: 8.0473 - val_loss: 7.0115
Epoch 7/250
 - 3s - loss: 7.8565 - val_loss: 6.9064
Epoch 8/250
 - 3s - loss: 7.6692 - val_loss: 6.7952
Epoch 9/250
 - 3s - loss: 7.4789 - val_loss: 6.6350
Epoch 10/250
 - 3s - loss: 7.2685 - val_loss: 6.4510
Epoch 11/250
 - 3s - loss: 7.0454 - val_loss: 6.2449
Epoch 12/250
 - 3s - loss: 6.8204 - val_loss: 6.0167
Epoch 13/250
 - 3s - loss: 6.5661 - val_loss: 5.7488
Epoch 14/250
 - 3s - loss: 6.2735 - val_loss: 5.4798
Epoch 15/250
 - 3s - loss: 6.0073 - val_loss: 5.3205
Epoch 16/250
 - 3s - loss: 5.8403 - val_loss: 5.2672
Epoch 17/250
 - 3s - loss: 5.7624 - val_loss: 5.2611
Epoch 18/250
 - 3s - loss: 5.7266 - val_loss: 5.2916
Epoc

Epoch 154/250
 - 3s - loss: 5.2705 - val_loss: 5.1107
Epoch 155/250
 - 3s - loss: 5.2816 - val_loss: 5.0791
Epoch 156/250
 - 3s - loss: 5.2736 - val_loss: 5.1062
Epoch 157/250
 - 3s - loss: 5.2802 - val_loss: 5.0930
Epoch 158/250
 - 3s - loss: 5.2636 - val_loss: 5.0799
Epoch 159/250
 - 3s - loss: 5.2760 - val_loss: 5.0752
Epoch 160/250
 - 3s - loss: 5.2873 - val_loss: 5.0659
Epoch 161/250
 - 3s - loss: 5.2644 - val_loss: 5.0920
Epoch 162/250
 - 3s - loss: 5.2683 - val_loss: 5.0746
Epoch 163/250
 - 3s - loss: 5.2717 - val_loss: 5.1585
Epoch 164/250
 - 3s - loss: 5.2705 - val_loss: 5.0719
Epoch 165/250
 - 3s - loss: 5.2717 - val_loss: 5.0760
Epoch 166/250
 - 3s - loss: 5.2616 - val_loss: 5.5715
Epoch 167/250
 - 3s - loss: 5.3006 - val_loss: 5.0640
Epoch 168/250
 - 3s - loss: 5.2673 - val_loss: 5.0719
Epoch 169/250
 - 3s - loss: 5.2777 - val_loss: 5.0906
Epoch 170/250
 - 3s - loss: 5.2608 - val_loss: 5.0705
Epoch 171/250
 - 3s - loss: 5.2676 - val_loss: 5.0876
Epoch 172/250
 - 3s - loss: 